In [1]:
import math
def pdf(mean,ssd,x):
    '''
    mean  均值
    ssd   标准差
    x     待测的数据
    计算的公式   概率密度函数
    '''
    ex=-(x-mean)**2/(2*ssd**2)
    epart=math.pow(math.e,ex)
    left=1.0/(math.sqrt(2*math.pi)*ssd)
    return left*epart

In [2]:
pdf(72.875,9.804,132)

5.152283971078022e-10

In [7]:
class Classifer:
    def __init__(self,filename,dataFormat):
        '''
        filename  文件名
        dataFormat 数据文件格式列表
        '''
        total=0
        classes={}  #存类别出现的次数  为先验概率做准备
        counts={}   #每个类别对应的属性值出现的次数  为条件概率做准备
        self.format=dataFormat.strip().split('\t')  #表示格式  数组
        self.prior={}  #先验概率
        self.condition={}  #条件概率
        
        
        #对数值的数据，因为计算概率密度函数
        totals={}
        numericValues={}
        
        #构建文件名
        f=open(filename,'r')
        lines=f.readlines()
        f.close()
        
        colValueCount={}    #{1:2,2:2,3:2,....}
        colValues={}
        
        for line in lines:
            total=total+1  #总记录数
            fields=line.strip().split('\t')   #切分每一条数据
            
            vector=[]   #存原始数据的位置
            nums=[]     #存数值型  字段值  用于概率密度函数运算
            for i in range( len(fields) ):
                #判断这一列是什么类型的值
                if self.format[i]=='attr':
                    vector.append(fields[i])
                elif self.format[i]=='class':
                    category=fields[i]
                elif self.format[i]=='num':
                    nums.append( float(fields[i]) )
                    
            #记录每个类别出现的次数
            classes.setdefault(category,0)
            counts.setdefault( category,{})
            #类别数加1
            classes[category]+=1  #{'i100':1,'i500':1}
            
            #处理每条记录出现的属性值
            #循环vector 取出每列值
            col=0
            for columnValue in vector:
                col+=1
                counts[category].setdefault(col,{} )
                counts[category][col].setdefault(columnValue,0 )  #{'i100':{1:{'both':1,'health':1,'apreace':1},2:{'sedentary':1}  }  }
                counts[category][col][columnValue]+=1
                colValues.setdefault(col,set() )
                colValues[col].add(columnValue)
                
            '''
                数值型的数据准备  totals={} 每种分类对应的值的和
                                  numericValues   每种分类对应的列的值的列表
            '''
            totals.setdefault(category,{})
            numericValues.setdefault(category,{})
            col=0
            for columnValue in nums:
                col+=1
                totals[category].setdefault(col,0)
                totals[category][col]+=columnValue
                numericValues[category].setdefault(col,[])
                numericValues[category][col].append(columnValue)
        
                
        for (col,columnValues)in colValues.items():
            colValueCount.setdefault(col,len(columnValues ) )
            print('第',col,'列有',len(columnValues),'个取值')
                
        #对应的计算成概率        
        #1.先验概率
        for (category,count) in classes.items():
            self.prior[category]=count/total
            
        #2.条件概率   注意 colums是一个字典
        for (category,columns) in counts.items():
            self.condition.setdefault(category,{})
            for (col,valueCount) in columns.items():
                self.condition[category].setdefault(col,{})
                for (attrValue,count) in valueCount.items():
                    #self.condition[category][col][attrValue] = count/classes[category]
                    #m=len(counts[category][col].items())
                    m=colValueCount[col]
                    print('分类',category,'第',col,'列的取值个数有:',m)
                    nc=count
                    n=classes[category]
                    temp=(1+nc)/(m+n)
                    self.condition[category][col][attrValue] = temp       
                    
        
        #连续值的条件概率运算
        #先完成 前提
            #计算出每个类别  (i100,i500)对应的这一列的均值
            #计算出每个类别  ( i100 ,i500)对应的样本标准差
            
        self.means={}
        self.totals=totals
        self.numericValues=numericValues
        #均值
        for (category,columnDict) in totals.items():
            self.means.setdefault(category,{})
            for (col,totalVal) in columnDict.items():
                self.means[category][col]=totalVal/len(numericValues[category][col])
        print('均值',self.means)
             
        #标准差
        self.ssd={}
        for (category,columnDict) in numericValues.items():
            self.ssd.setdefault(category,{})
            for (col,values) in columnDict.items():
                sumOfSquareDifference=0.0    #样本与均值差的平方和
                mean=self.means[category][col]
                for value in values:
                    sumOfSquareDifference+=(value-mean)**2
                self.ssd[category][col]=math.sqrt(sumOfSquareDifference/(len(values)-1)  )
        print('标准差:',self.ssd)
        
        
        
        
        #最后将  分类数和条件数值数存到self中
        self.classCounts=classes  #分类计数
        self.conditionCount=counts  #条件计数
        self.colValueCount=colValueCount
        
    def classify(self,itemVector,numVector):
        '''
        朴素贝叶斯：求出所有可能性的概率值，取大值
        '''
        self.result=[]  #[ (后验概率，类别1),(后验概率，类别2),.....]
        for (category,priors) in self.prior.items():
            prob=priors  #这个category类别的先验概率
            col=1
            for colValue in itemVector:
                if not colValue in self.condition[category][col]:
                    print('分类为:',category,'第',col,'列',colValue,'出现了空值')
                    m=len(self.condition[category][priors].items())
                    temp=1/(m+self.classCounts[category])
                    prob=prob*temp
                else:
                    prob=prob * self.condition[category][col][colValue]
                col+=1
                
                
            #继续计算连续数的概率密度函数
            col=1
            for numValue in numVector:
                #计算密度
                meanValue=self.means[category][col]
                ssdValue=self.ssd[category][col]
                pdfProb=pdf(meanValue,ssdValue,numValue)
                prob=prob*pdfProb
                print(numValue,'分类',category,'第',col,'列pdf的值：',pdfProb)
                col+=1
            
            self.result.append( (prob,category) )  # [(0.3,'i100'), ]
        totalProb=0.0
        print(self.result)
        for (p,cat) in self.result:
            totalProb+=p
            
        self.belief=max(self.result)[0]/totalProb
        return max(self.result)[1]

In [8]:
a=Classifer('i-01','attr\tattr\tattr\tattr\tnum\tclass')
print(a.classCounts)
print(a.conditionCount)

#模型
print(a.prior)
print(a.condition)

第 1 列有 3 个取值
第 2 列有 3 个取值
第 3 列有 2 个取值
第 4 列有 2 个取值
分类 i100 第 1 列的取值个数有: 3
分类 i100 第 1 列的取值个数有: 3
分类 i100 第 1 列的取值个数有: 3
分类 i100 第 2 列的取值个数有: 3
分类 i100 第 2 列的取值个数有: 3
分类 i100 第 2 列的取值个数有: 3
分类 i100 第 3 列的取值个数有: 2
分类 i100 第 3 列的取值个数有: 2
分类 i100 第 4 列的取值个数有: 2
分类 i100 第 4 列的取值个数有: 2
分类 i500 第 1 列的取值个数有: 3
分类 i500 第 1 列的取值个数有: 3
分类 i500 第 1 列的取值个数有: 3
分类 i500 第 2 列的取值个数有: 3
分类 i500 第 2 列的取值个数有: 3
分类 i500 第 2 列的取值个数有: 3
分类 i500 第 3 列的取值个数有: 2
分类 i500 第 3 列的取值个数有: 2
分类 i500 第 4 列的取值个数有: 2
分类 i500 第 4 列的取值个数有: 2
均值 {'i100': {1: 70.83333333333333}, 'i500': {1: 106.11111111111111}}
标准差: {'i100': {1: 16.557978942693058}, 'i500': {1: 21.327473153449542}}
{'i100': 6, 'i500': 9}
{'i100': {1: {'both': 3, 'appearance': 2, 'health': 1}, 2: {'sedentary': 3, 'moderate': 1, 'active': 2}, 3: {'moderate': 5, 'aggressive': 1}, 4: {'yes': 2, 'no': 4}}, 'i500': {1: {'health': 4, 'appearance': 3, 'both': 2}, 2: {'sedentary': 2, 'active': 4, 'moderate': 3}, 3: {'moderate': 3, 'aggressive': 6}, 4: {'yes': 6, 'n

In [9]:
#  健康  锻炼适中  动机中等  习惯于技术设备
print('推荐购买的类型为:',a.classify(['health','moderate','moderate','yes'],[100]))

print(a.result)
print('以上结果的置信度:',a.belief)

100 分类 i100 第 1 列pdf的值： 0.005106581556764106
100 分类 i500 第 1 列pdf的值： 0.017953210304530804
[(2.8369897537578367e-05, 'i100'), (0.00034620515738764636, 'i500')]
推荐购买的类型为: i500
[(2.8369897537578367e-05, 'i100'), (0.00034620515738764636, 'i500')]
以上结果的置信度: 0.9242611135884589
